# MIS: Clase Complementaria 17 (2023-20)

El día de hoy nos centraremos en el entendimiento del planteamiento de un algoritmo que permite calcular los caminos más cortos de un nodo a otro.

In [1]:
# Importar paquetes relevantes
import numpy as np
import networkx as nx
import itertools as it
import random

# Importar la librería de Google OR-Tools
from ortools.linear_solver import pywraplp

A continuación se presenta un ejemplo de transporte óptimo. A partir del ejemplo se desarrollará una actividad en clase.

El problema:

MG Auto cuenta con tres plantas en Los Ángeles, Detroit y Nueva Orleáns, y dos importantes centros de distribución en Denver y Miami. Las capacidades trimestrales de las tres plantas son 1000, 1500 y 1200 automóviles, y las demandas de los dos centros de distribución durante el mismo periodo son de 2300 y 1400 automóviles. La distancia en millas entre las plantas y los centros de distribución aparece en la siguiente tabla:


| Ciudades    | Denver | Miami |
|---:|---:|---:|
|      LA     |  1000  |  2690 |
|   Detroit   |  1250  |  1350 |
| New Orleans |  1275  |   850 |



La siguiente imagen representa el problema previo, mostrando las demandas y ofertas de cada una de las ciudades.


![Image2](https://www.ingenieriaindustrialonline.com/wp-content/uploads/2021/08/Figure-2021-08-10-093443-1.png)

In [2]:
# Ejemplo de transporte óptimo.
# Código tomado de https://www.ingenieriaindustrialonline.com/investigacion-de-operaciones/solucion-de-un-modelo-de-transporte-mediante-un-algoritmo-de-asignacion/

# Declarar el solucionador que abordará el modelo
solver = pywraplp.Solver.CreateSolver('SCIP')

# Data del modelo
costos = [
    [ 80, 215],
    [100, 108],
    [102,  68],
]

plantas = ['Los Ángeles', 'Detroit', 'New Orleans']
cedis = ['Denver', 'Miami']

num_plantas = len(costos)
num_cedis = len(costos[0])

# Variables del modelo
x = {}
for i in range(num_plantas):
    for j in range(num_cedis):
        x[i, j] = solver.IntVar(0, solver.infinity(), '')
        
#Restricciones de disponibilidad (oferta en plantas) 
solver.Add(solver.Sum([x[0, j] for j in range(num_cedis)]) <= 1000) 
solver.Add(solver.Sum([x[1, j] for j in range(num_cedis)]) <= 1500) 
solver.Add(solver.Sum([x[2, j] for j in range(num_cedis)]) <= 1200) 

#Restricciones de demanda (cedis) 
solver.Add(solver.Sum([x[i, 0] for i in range(num_plantas)]) >= 2300) 
solver.Add(solver.Sum([x[i, 1] for i in range(num_plantas)]) >= 1400)      



# Función objetivo con criterio de optimización: minimizar
objective_terms = []
for i in range(num_plantas):
    for j in range(num_cedis):
        objective_terms.append(costos[i][j] * x[i, j])

solver.Minimize(solver.Sum(objective_terms))

# Invoca el solucionador
status = solver.Solve()

# Configura los parámetros de impresión, las salidas del modelo
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print('Costo total = ', solver.Objective().Value(), '\n')

    for i in range(num_plantas):
        for j in range(num_cedis):
            print('|{:^20} -> {:^20} | Cantidad: {:^20}|'.format(
            plantas[i],
            cedis[j],
            x[i, j].solution_value())) 



Costo total =  313200.0 

|    Los Ángeles      ->        Denver        | Cantidad:        1000.0       |
|    Los Ángeles      ->        Miami         | Cantidad:         0.0         |
|      Detroit        ->        Denver        | Cantidad:        1300.0       |
|      Detroit        ->        Miami         | Cantidad:        200.0        |
|    New Orleans      ->        Denver        | Cantidad:         0.0         |
|    New Orleans      ->        Miami         | Cantidad:        1200.0       |


En la siguiente imagen se pueden apreciar las asignaciones finales de la oferta disponible
 
![Image2](https://www.ingenieriaindustrialonline.com/wp-content/uploads/2021/08/modelo-de-transporte-solucion-de-red.png)

A continuación implemente una función que reciba un grafo con pesos (atributo de enlaces llamado "costos") de ir de un conjunto de nodos "O" (oferentes) a un conjunto de nodos "D" (demandantes). Además, el grafo contiene la clasificación del nodo (atributo de nodo llamado "tipo", el cuál es "O" u "D") y un valor de Oferta/Demanda (atributo de nodo llamado "valor"). La función debe imprimir la distribución de la oferta entre los demandantes y retornar un grafo que tenga como únicas conexiones las que se establecen entre oferentes que venden una cantidad positiva a un demandante (en un atributo de enlace llamado "transferencia"). 

La estructura de la función es transporte_optimo(grafo:nx.DiGraph)->nx.DiGraph. Finalmente, para que esta implementación funcione asegurese que la oferta agregada es mayor o igual a la demanda agregada. Si este no es el caso, su función debe levantar una excepción que diga "Imposible realizar la optimización".     

In [3]:
def asignador_tipo(i,umbral):
    tipo = 'D'
    if i<umbral:
        tipo = 'O'
    return tipo

def asignador_enlaces(i,j,umbral):
    asignado = False
    if (i<umbral) and (j>=umbral):
        asignado = True 
    return asignado

def asignador_costos(a,b):
    asignado = random.randint(a,b)
    return asignado

In [4]:
# Grafo para hacer pruebas:

# Básicos
N = 10 # Número de agentes
umbral = (N//4)*3 # Umbral que separa los ids de los oferentes (<umbral) y los demandantes (>=umbral)
a = 20 # Límite inferior del costo
b = 80 # Límite superior del costo
alpha = 1 # \alpha\in (0,1] Relación aproximada máxima de Demanda/Oferta 

# Creación del grafo
G = nx.DiGraph()
G.add_nodes_from(range(N))
G.add_edges_from([(i,j) for i,j in it.product(G.nodes,G.nodes) if asignador_enlaces(i,j,umbral)])

# Atributo de tipo de agente
dict_tipo = {i:asignador_tipo(i,umbral) for i in G.nodes}
nx.set_node_attributes(G,dict_tipo,'tipo')

# Atributo de costo de los enlaces
dict_costos = {enlace:asignador_costos(a,b) for enlace in G.edges}
nx.set_edge_attributes(G,dict_costos,'costos')

# Atributo de valor
A = np.floor(a*(N/umbral))
B = np.floor(b*(N/umbral))
dict_D = {i:asignador_costos(A,B) for i in G.nodes if G.nodes[i]['tipo']=='D'}
dict_O = {i:asignador_costos(a,b) for i in G.nodes if G.nodes[i]['tipo']=='O'}
c = 0
oferta = sum(dict_O.values())
for nodo in dict_D:
    if c+dict_D[nodo]>alpha*oferta:
        dict_D[nodo] = 0
    c += dict_D[nodo]
dict_valores = {}

for diccionario in [dict_O,dict_D]:
    for nodo in diccionario:
        dict_valores[nodo] = diccionario[nodo]

nx.set_node_attributes(G,dict_valores,'valor')

In [5]:
# Pueden revisar esto para ver los atributos

#nx.get_edge_attributes(G,'costos')
#nx.get_node_attributes(G,'tipo')
nx.get_node_attributes(G,'valor')

{0: 68, 1: 58, 2: 43, 3: 79, 4: 43, 5: 74, 6: 85, 7: 38, 8: 114, 9: 128}

In [9]:
# Espacio para la actividad en clase:
   
    